In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')   

In [5]:
def count_pass_fail_ratio(master) :
    master = master.rename(columns={'Ts_User_08_(Test_ID)': 'test_id'})
    p4 = master.groupby(['test_id','Rn_Status'])
    df = p4.Ts_Name.agg('count').reset_index()
    df = df.rename(columns={'Ts_Name': 'count'})
    df = df[df['count'] > 0]
    pass_df = df[df['Rn_Status'] == 'Passed']
    fail_df = df[df['Rn_Status'] == 'Failed']
    df4 = pd.merge(pass_df,fail_df,how='outer',on=['test_id'])
    df4 = df4.rename(columns={'count_x': 'pass_count','count_y':'fail_count'})
    df4['pass_count'].fillna(0,inplace=True)
    df4['fail_count'].fillna(0,inplace=True)
    df4['pass_ratio'] = df4['pass_count']/(df4['pass_count']+ df4['fail_count'])
    df4.drop(labels=['Rn_Status_x', 'Rn_Status_y'], axis=1,inplace=True)
    master = pd.merge(master,df4,how='left',on=['test_id'])
    master = master.rename(columns={'test_id':'Ts_User_08_(Test_ID)'})
    return master

In [6]:
TC = pd.read_excel('Test_results_for_POC_21092018.xlsx',parse_dates=[['Rn Execution Date', 'Rn Execution Time']])
TC = TC.rename(columns={'Rn Execution Date_Rn Execution Time':'date'})
TC = count_pass_fail_ratio(TC)
Defects = pd.read_excel('Defect_results_for_POC_21092018.xlsx')
df1 = pd.read_excel('Testcases_CTC_MS_for_POC_27092018.xlsx')
TC.columns = TC.columns.str.replace(' ', '_')
Defects.columns = Defects.columns.str.replace(' ', '_')
df1.columns = df1.columns.str.replace(' ', '_')
Defects = Defects.rename(columns={'Linked_Test_ID': 'Ts_User_08_(Test_ID)','Terminal':'Cy_Terminal_Name', 'Test_Area':'Ts_Test_Area'})
raw_data = pd.merge(TC,df1,how='outer',on=['Ts_Test_Area','Ts_User_08_(Test_ID)','Ts_Name','Ts_Executing_OpCos','Ts_Coverage','Ts_TCD_Releases'])
master = pd.merge(raw_data,Defects,how='outer',on=['Ts_User_08_(Test_ID)','Ts_Test_Area','Cy_Terminal_Name'])
print("Processing the previously read Data .......")
master['Cov_0'],master['Cov_1'],master['Cov_2'],master['Cov_3'],master['Cov_4'],master['Cov_5'],master['Cov_6'],master['Cov_7'] = master['Ts_Coverage'].str.split(';').str
master['Reason to select'] = ''
master = master.rename(columns={'Ts_User_08_(Test_ID)': 'test_id'})
master.to_excel("Alltestcases.xlsx",index=False)
print("All test cases generated")

Processing the previously read Data .......
All test cases generated


## Non- Optimized TC from rule engine

In [53]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')   
master = pd.read_excel('Alltestcases.xlsx')

In [54]:
def RN_Diff(diff_df):
    servicenameArray = []
    testname = []
    dict={}
    filter_col = [col for col in diff_df if 'DefectID' in col]
    DefectArray=diff_df[filter_col].iloc[0].dropna().values
    filter_col = [col for col in diff_df if 'Bug Id' in col]
    KnownissueArray=diff_df[filter_col].iloc[0].dropna().values
    release = [diff_df['RH-Latest-RelName'].values[0]]
    if diff_df["GenInfo-IMEI-SV"].values[0] != diff_df["GenInfo-IMEI-SV"].values[1]:
        testname = ['IMEI-SV']
    for i in range(1,11):
        statuslevel='FC-' +str(i)+ '-Status-level'
        servicename='FC-' +str(i)+ '-ServiceName'
        if(diff_df[statuslevel].iloc[1] == ('Complete' or 'Completed')) :
            if(diff_df[statuslevel].iloc[0]!=diff_df[statuslevel].iloc[1]): 
                servicenameArray.append(diff_df[servicename].values[0])
    terminal = diff_df['PD-ProjectName'].values[0]
    dict={'Defect':DefectArray,'known_issue':KnownissueArray,'service_name':servicenameArray,'imei_sv':testname,
     'release_type':release,'terminal':terminal} 
    print("dict ",dict)
    TCFinal= order_filter(dict)
    return (TCFinal)
#RN_Diff(diff_df)

In [55]:
def join_rows(df) :
    new_df = pd.DataFrame(0, index=range(1),columns = df.columns)
    for i in range(df.shape[1]) :                       
        temp = set()
        for j in range(df.shape[0]) :
            if j == 0:
                new_df.iloc[j,i] = df.iloc[j,i]
            else :
                if df.iloc[j,i] != new_df.iloc[0,i] :
                    temp.add(str(df.iloc[j,i]))
        if (len(temp) > 0) :
            temp.add(str(new_df.iloc[0,i]))
            if 'nan' in temp :
                temp.remove('nan')
            new_df.iloc[0,i] = ', '.join(temp)
    return new_df

In [56]:
def format_excel(final,final_nonOptimized) :
    writer = pd.ExcelWriter("FinalTestSuite.xlsx", engine='xlsxwriter')
    final.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False,index=False)
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    worksheet.set_column('A:AA', 25)
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#FFCCCC',
        'border': 1})
    for col_num, value in enumerate(final.columns.values):
        worksheet.write(0, col_num, value, header_format)
    writer.save()
    
    writer = pd.ExcelWriter("FinalTestSuite_nonOptimized.xlsx", engine='xlsxwriter')
    final_nonOptimized.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False,index=False)
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    worksheet.set_column('A:AN', 25)
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#FFCCCC',
        'border': 1})
    for col_num, value in enumerate(final_nonOptimized.columns.values):
        worksheet.write(0, col_num, value, header_format)
    writer.save()

In [57]:
def order_filter(filterparams):
    FinalTC = pd.DataFrame()
    FinalTC_nonOptimized = pd.DataFrame()
    TCexcel1 = pd.DataFrame()
    TCexcel2 = pd.DataFrame()
    TCexcel3 = pd.DataFrame()
    TCexcel4 = pd.DataFrame()
    TCexcel5 = pd.DataFrame()
    areas = set(['Exploratory A','Smart_Customization','VoWiFi','MMS'])
    for key,value in filterparams.items():
        if key == 'Defect':
            if(len(filterparams[key]) != 0) :
                TCexcel1 = Defects_ID (filterparams[key])
        elif key == 'release_type':
            if(len(filterparams[key]) != 0) :
                TCexcel2 = Terminal_Release(filterparams[key])
                TCexcel4 = Terminal_Release_nonOptimized(filterparams[key])
        elif key == 'service_name':
            areas.update(filterparams[key])
        elif key == 'test_area':
            areas.update(filterparams[key])
        elif key == 'terminal':
            Terminal_name = filterparams[key]
    TCexcel3 = Test_area(list(areas))
    TCexcel5 = Test_area_nonOptimized(list(areas))
    FinalTC = FinalTC.append([TCexcel1,TCexcel2,TCexcel3])
    FinalTC_nonOptimized = FinalTC_nonOptimized.append([TCexcel1,TCexcel4,TCexcel5])
    
    FinalTC = FinalTC[(FinalTC['Cy_Terminal_Name'] == Terminal_name) | (FinalTC['Cy_Terminal_Name'] =='Blanc')] 
    FinalTC.dropna(subset=['Ts_Name','Defect_ID'],inplace=True)
    FinalTC.drop(labels=['Cov_0', 'Cov_1', 'Cov_2', 'Cov_3','Cov_4', 'Cov_5', 'Cov_6', 'Cov_7','Rn_Tester_Name','Ts_Description(draft)'
                        ,'Full_Defect_Comments','Detected_By'],axis=1,inplace=True)
    FinalTC.drop_duplicates(keep='first',inplace=True)
    group = FinalTC.groupby(['Ts_Name'])
    final = pd.DataFrame()
    final = final.append(group.apply(join_rows))
    final = final[['Cy_Terminal_Name', 'Reason to select', 'Ts_Test_Area','test_id','Ts_Name', 'Description', 
                   'Ts_Suggested_Severity', 'Cy_Terminal_Release', 'Cy_Terminal_SW_Version','Cy_Executing_OpCo', 
       'Ts_TCD_Releases', 'Ts_Executing_OpCos', 'Ts_Coverage', 'Rn_Status', 'Scope_CTC', 'Scope_CZ', 'Scope_DE', 'Scope_HU', 
                   'Scope_IE', 'Scope_NL', 'Scope_UK',  'Defect_ID', 'Summary','Status', 'Severity', 'Also_in_OpCo']]
    final = final.rename(columns={'Description': 'Test Description','Summary':'Defect Summary'})
    
    # final tc non optimized
    FinalTC_nonOptimized = FinalTC_nonOptimized[(FinalTC_nonOptimized['Cy_Terminal_Name'] == Terminal_name) | (FinalTC_nonOptimized['Cy_Terminal_Name'] =='Blanc')] 
    FinalTC_nonOptimized.dropna(subset=['Ts_Name'],inplace=True)
    FinalTC_nonOptimized.drop(labels=['Cov_0', 'Cov_1', 'Cov_2', 'Cov_3','Cov_4', 'Cov_5', 'Cov_6', 'Cov_7','Rn_Tester_Name','Ts_Description(draft)'
                        ,'Full_Defect_Comments','Detected_By'],axis=1,inplace=True)
    FinalTC_nonOptimized.drop_duplicates(keep='first',inplace=True)
    group = FinalTC_nonOptimized.groupby(['Ts_Name'])
    final_nonOptimized = pd.DataFrame()
    final_nonOptimized = final_nonOptimized.append(group.apply(join_rows))
    final_nonOptimized = final_nonOptimized[['Cy_Terminal_Name', 'Reason to select', 'Ts_Test_Area','test_id','Ts_Name', 'Description', 
                   'Ts_Suggested_Severity', 'Cy_Terminal_Release', 'Cy_Terminal_SW_Version','Cy_Executing_OpCo', 
       'Ts_TCD_Releases', 'Ts_Executing_OpCos', 'Ts_Coverage', 'Rn_Status', 'Scope_CTC', 'Scope_CZ', 'Scope_DE', 'Scope_HU', 
                   'Scope_IE', 'Scope_NL', 'Scope_UK',  'Defect_ID', 'Summary','Status', 'Severity', 'Also_in_OpCo']]
    final_nonOptimized = final_nonOptimized.rename(columns={'Description': 'Test Description','Summary':'Defect Summary'})
    
    format_excel(final,final_nonOptimized)
    
    print('Processed Successfully -Final Test case sheet is available at: {0}'.format(os.getcwd()))
    return final 
#tc = order_filter(dict)

In [58]:
def Terminal_Release(release_name):
    TCexcel = pd.DataFrame()
    cycle_type = 'SH'
    for rel in release_name:
        TCexcel = TCexcel.append(master.loc[master['Cy_Terminal_Release'].str.slice(0,3) == rel ]) 
        TCexcel= TCexcel[(TCexcel['Cov_1']==cycle_type) | (TCexcel['Cov_2']==cycle_type) | 
                                           (TCexcel['Cov_3']==cycle_type) |(TCexcel['Cov_4']==cycle_type) |
                                           (TCexcel['Cov_5']==cycle_type)|(TCexcel['Cov_6']==cycle_type)|
                         (TCexcel['Cov_7']==cycle_type)|(TCexcel['Cov_0']==cycle_type)]
    TCexcel = TCexcel[(TCexcel['Ts_Suggested_Severity']== '2 - Serious') | (TCexcel['Ts_Suggested_Severity']== '1a - Fatal') | 
                                           (TCexcel['Ts_Suggested_Severity']== '3 - Major') |(TCexcel['Rn_Status']== 'Failed')|
                      (TCexcel['Status'] == 'Closed' )]
    TCexcel = TCexcel[(TCexcel['Rn_Status']== 'Failed') | ((TCexcel['Rn_Status']== 'Passed') & (TCexcel['pass_ratio'] < 0.8 ))]
    TCexcel['Reason to select'] = np.where(TCexcel['Cy_Terminal_Release'].str.slice(0,3)== rel, 'Terminal release is :{0} hence cycle type  {1}'.format(rel, cycle_type),'none')
    #excel = TCexcel.to_excel('TC_for_execution_Release_based.xlsx')
    return (TCexcel)
#t = Terminal_Release(['MR0'])

In [59]:
def Terminal_Release_nonOptimized(release_name):
    TCexcel = pd.DataFrame()
    cycle_type = 'SH'
    for rel in release_name:
        TCexcel = TCexcel.append(master.loc[master['Cy_Terminal_Release'].str.slice(0,3) == rel ]) 
        TCexcel= TCexcel[(TCexcel['Cov_1']==cycle_type) | (TCexcel['Cov_2']==cycle_type) | 
                                           (TCexcel['Cov_3']==cycle_type) |(TCexcel['Cov_4']==cycle_type) |
                                           (TCexcel['Cov_5']==cycle_type)|(TCexcel['Cov_6']==cycle_type)|
                         (TCexcel['Cov_7']==cycle_type)|(TCexcel['Cov_0']==cycle_type)]
    TCexcel = TCexcel[(TCexcel['Ts_Suggested_Severity']== '2 - Serious') | (TCexcel['Ts_Suggested_Severity']== '1a - Fatal') | 
                                           (TCexcel['Ts_Suggested_Severity']== '3 - Major') |(TCexcel['Rn_Status']== 'Failed')|
                      (TCexcel['Status'] == 'Closed' )]
    TCexcel['Reason to select'] = np.where(TCexcel['Cy_Terminal_Release'].str.slice(0,3)== rel, 'Terminal release is :{0} hence cycle type  {1}'.format(rel, cycle_type),'none')
    #excel = TCexcel.to_excel('TC_for_execution_Release_based_nonOptimized.xlsx')
    return (TCexcel)

In [60]:
def Test_area(testarea):
    TCexcel = pd.DataFrame()
    cycle_type = 'OE'
    for area in testarea:
        if (area =='Exploratory A'):
            cycle_type = 'SH'
        df = master.loc[master['Ts_Test_Area'] == area]
        df['Reason to select'] = 'Change in version/status of test area :{0} hence cycle type  {1}'.format(area, cycle_type) 
        TCexcel = TCexcel.append(df)
        TCexcel= TCexcel[(TCexcel['Cov_1']==cycle_type) | (TCexcel['Cov_2']==cycle_type) | 
                                           (TCexcel['Cov_3']==cycle_type) |(TCexcel['Cov_4']==cycle_type) |
                                           (TCexcel['Cov_5']==cycle_type)|(TCexcel['Cov_6']==cycle_type)|
                                           (TCexcel['Cov_7']==cycle_type)|(TCexcel['Cov_0']==cycle_type)]
        if cycle_type == 'SH':
            TCexcel= TCexcel[(TCexcel['Ts_Suggested_Severity']== '2 - Serious') | (TCexcel['Ts_Suggested_Severity']== '1a - Fatal') | 
                                           (TCexcel['Ts_Suggested_Severity']== '3 - Major') |(TCexcel['Rn_Status']== 'Failed')]
    TCexcel = TCexcel[(TCexcel['Rn_Status']== 'Failed') | ((TCexcel['Rn_Status']== 'Passed') & (TCexcel['pass_ratio'] < 0.8 ))]
    #TCexcel.to_excel('area.xlsx',index=False)
    return (TCexcel)
#t = Test_area(['Exploratory A','IMEI-SV'])

In [61]:
def Test_area_nonOptimized(testarea):
    TCexcel = pd.DataFrame()
    cycle_type = 'OE'
    print(testarea)
    for area in testarea:
        if (area =='Exploratory A'):
            cycle_type = 'SH'
        df = master.loc[master['Ts_Test_Area'] == area]
        df['Reason to select'] = 'Change in version/status of test area :{0} hence cycle type  {1}'.format(area, cycle_type) 
        TCexcel = TCexcel.append(df)
        TCexcel= TCexcel[(TCexcel['Cov_1']==cycle_type) | (TCexcel['Cov_2']==cycle_type) | 
                                           (TCexcel['Cov_3']==cycle_type) |(TCexcel['Cov_4']==cycle_type) |
                                           (TCexcel['Cov_5']==cycle_type)|(TCexcel['Cov_6']==cycle_type)|
                                           (TCexcel['Cov_7']==cycle_type)|(TCexcel['Cov_0']==cycle_type)]
        if cycle_type == 'SH':
            TCexcel= TCexcel[(TCexcel['Ts_Suggested_Severity']== '2 - Serious') | (TCexcel['Ts_Suggested_Severity']== '1a - Fatal') | 
                                           (TCexcel['Ts_Suggested_Severity']== '3 - Major') |(TCexcel['Rn_Status']== 'Failed')]
    return (TCexcel)
#check12 = Test_area_nonOptimized(["Exploratory A","3G_Datacard"])

In [62]:
def Defects_ID (defectid):
    TCexcel = pd.DataFrame()
    TCexcel1 = pd.DataFrame()
    for ids in defectid:
        cycle_type = 'SH'
        sev_check =  set(master.loc[master['Defect_ID'] == ids]['Severity'])
        for sev in sev_check :
            if(sev in ['4 - Minor' , '3 - Major'] ):
                cycle_type = 'BF'
        TCarea = set(master.loc[master['Defect_ID'] == ids]['Ts_Test_Area'])
        TCname = set(master.loc[master['Defect_ID'] == ids]['Ts_Name'])
        TCopco = set(master.loc[master['Defect_ID'] == ids]['Ts_Executing_OpCos'])
        for area in TCarea :
            df = master.loc[master['Ts_Test_Area'] == area]
            df['Reason to select'] = 'TC belongs to area:{0} for defect id:{1}'.format(area, ids)
            TCexcel = TCexcel.append(df)
            TCexcel= TCexcel[(TCexcel['Cov_1']==cycle_type) | (TCexcel['Cov_2']==cycle_type) | 
                                           (TCexcel['Cov_3']==cycle_type) |(TCexcel['Cov_4']==cycle_type) |
                                           (TCexcel['Cov_5']==cycle_type)|(TCexcel['Cov_6']==cycle_type)|
                                           (TCexcel['Cov_7']==cycle_type)|(TCexcel['Cov_0']==cycle_type)]
            TCexcel = TCexcel.append(master.loc[master['Ts_Executing_OpCos'] == TCopco])
        if cycle_type == 'SH':
            TCexcel= TCexcel[(TCexcel['Ts_Suggested_Severity']== '2 - Serious') | (TCexcel['Ts_Suggested_Severity']== '1a - Fatal') | 
                                           (TCexcel['Ts_Suggested_Severity']== '3 - Major') |(TCexcel['Rn_Status']== 'Failed')]
        elif cycle_type == 'BF':
            TCexcel= TCexcel[TCexcel['Rn_Status']== 'Failed']
    
        for name in TCname:
            if name :
                if (TCexcel['Ts_Name'].any() == name ) :
                    TCexcel['Reason to select'] = np.where(TCexcel['Ts_Name'] == name, 'linked TC for defect id:{0}'.format(ids),'none')
                else : 
                    TCexcel = TCexcel.append( master[ (master['Ts_Name'] == name) & (master['Defect_ID'] == ids)])
                    #TCexcel['Reason to select'] = (TCexcel['Ts_Name'] == name, 'linked TC for defect id:{0}'.format(ids),'none')
                    TCexcel.loc[TCexcel['Ts_Name'] == name, 'Reason to select'] = 'linked TC for defect id:{0}'.format(ids)    
        TCexcel1 = TCexcel1.append(TCexcel)
    #excel = TCexcel1.to_excel('TC_for_execution_defect_based.xlsx')
    return (TCexcel1)
#t = TCexcel = Defects_ID ([24093, 23993, 25649])
#t.to_excel('defect_id.xlsx')

In [63]:
def take_second_release_notes(diff_df) :
    file2 = input('Please enter excel file names of previous release : ' )
    if(not (file2.endswith('.xls') or file2.endswith('.xlsx'))) :
        file2 = file2+'.xlsx'
    if(not(os.path.isfile(file2))) :
        print('no file found for given name : '+file2)
        take_second_release_notes(diff_df)
    row2 = pd.read_excel(file2)
    print(diff_df['RH-Latest-RelName'])
    if(row2.empty) :
        print('Given excel file is empty')
        take_second_release_notes(diff_df)
    if(row2.duplicated(subset=' Release Notes Name').any() == True) :
        print('Given file contains duplicate column names.Please update it.')
        take_second_release_notes(diff_df)
    diff_df = diff_df.append(row2.set_index(' Release Notes Name').T)
    RN_Diff(diff_df)

In [64]:
def take_latest_release_notes() :
    file1 = input('Please enter excel file names of latest release : ' )       
    if(not (file1.endswith('.xls') or file1.endswith('.xlsx'))) :
        file1 = file1+'.xlsx'
    if(not(os.path.isfile(file1))) :
        print('no file found for given name : '+file1)
        take_latest_release_notes()
    row1 = pd.read_excel(file1)
    if(row1.empty) :
        print('Given excel file is empty')
        take_latest_release_notes()
    if(row1.duplicated(subset=' Release Notes Name').any() == True) :
        print('Given file contains duplicate column names.Please update it.')
        take_latest_release_notes()
    df = row1.set_index(' Release Notes Name').T
    take_second_release_notes(df)        

In [65]:
take_latest_release_notes()

Please enter excel file names of latest release : Release notes_Hima_TA2_DSIM
Please enter excel file names of previous release : Release notes_Hima_TA1_DSIM
HUAWEI  HMA-L29 9.0.0.108(C432E10R1P16) for COTA-Normal Release Notes    TA
Name: RH-Latest-RelName, dtype: object
dict  {'Defect': array([25731, 25706, 25717, 25724], dtype=object), 'known_issue': array([25673, 25711, 25713, 25718, 25719, 25721, 25722, 25728, 25737],
      dtype=object), 'service_name': ['Advanced LTE DL'], 'imei_sv': ['IMEI-SV'], 'release_type': ['TA'], 'terminal': 'Hima'}
['Advanced LTE DL', 'Exploratory A', 'VoWiFi', 'MMS', 'Smart_Customization']
Processed Successfully -Final Test case sheet is available at: /Users/natashapandya/Desktop/Final/rule engine model


### Notes

Release notes_Hima_MRO_DSIM
dict  {'Defect': [25670, 25717, 25719, 25722, 25728],'service_name': [], 'imei_sv': ['IMEI-SV'], 'release_type': ['MR0'], 'terminal': 'Hima'}

Release notes_Hima_TA2_DSIM
dict  {'Defect': ([25731, 25706, 25717, 25724],service_name': ['Advanced LTE DL'], 'imei_sv': ['IMEI-SV'], 'release_type': ['TA'], 'terminal': 'Hima'}